# Reversal / Momentum - Time Horizon
In this homework, we explore how reversal tends to exist at shorter horizons and momentum at longer horizons. We do this on 4h cryptocurrency price data. Please first run the code below, which will download the price data (from Binance since 01/2020 to 12/2022) and compute returns based on it, stored in `ret`.

1. Use `ret` to generate rank-demeaned-normalized XS reversal strategies at 4,8,12,16,20, and 24 hour frequencies.
2. IE. for the 12 hour strategy, you will use the average return in the last three 4-hour bars to form a rank-demeaned-noramlzied XS portfolio and hold for the next 4 hours.
3. Compute the Sharpe ratios of the reversal strategy at these different horizons.
4. At what horizons do you observe reversal vs. momentum?
5. The first bar typically contains the most reversal. "Skip" the first bar by lagging your portfolio by one 4h bar (similar to how UMD does at the monthly frequency). This should strengthen any momentum you see.

In [ ]:
from binance.client import Client as bnb_client
from datetime import datetime
import pandas as pd 
import numpy as np 

#client = bnb_client()
###  if you're in the US, use: 
client = bnb_client(tld='US')#" here instead

def get_binance_px(symbol,freq,start_ts = '2020-01-01',end_ts='2022-12-31'):
    data = client.get_historical_klines(symbol,freq,start_ts,end_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    data['open_time'] = data['open_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    data['close_time'] = data['close_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    return data 

univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']

freq = '4h'
px = {}
for x in univ:
    data = get_binance_px(x,freq)
    px[x] = data.set_index('open_time')['close']

px = pd.DataFrame(px).astype(float)
px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq=freq))
ret = px.pct_change()

In [ ]:
strats = {}
for hor in [1,2,3,4,5,6]:
    avg_ret = ret.rolling(hor,min_periods=1).mean().rank(1)
    avg_ret = avg_ret.subtract(avg_ret.mean(1),0)
    avg_ret = avg_ret.divide(avg_ret.abs().sum(1),0)
    strats[hor] = (avg_ret.shift()*ret).sum(1)
strats = pd.DataFrame(strats)

We get reversal at horizons of <=8 hours. Momentum exists at longer horizons.

In [ ]:
sr = strats.mean()/strats.std()*np.sqrt(252*24/4)
sr

In [ ]:
sr = strats.resample('A').mean()/strats.resample('A').std()*np.sqrt(252*24/4)

In [ ]:
sr.plot(kind='bar')

In [ ]:
strats_lag = {}
for hor in [1,2,3,4,5,6]:
    avg_ret = ret.rolling(hor,min_periods=1).mean().rank(1)
    avg_ret = avg_ret.subtract(avg_ret.mean(1),0)
    avg_ret = avg_ret.divide(avg_ret.abs().sum(1),0)
    strats_lag[hor] = (avg_ret.shift(2)*ret).sum(1)
strats_lag = pd.DataFrame(strats_lag)

Skipping the most recent bar bumps up the momentum effect significantly, and we now see positive sharpes as high as ~3.

In [ ]:
sr_lag = strats_lag.mean()/strats_lag.std()*np.sqrt(252*24/4)
sr_lag